In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import *
conf = SparkConf().set('spark.driver.extraJavaOptions','-Dio.netty.tryReflectionSetAccessible=true')\
                  .set('spark.executor.extraJavaOptions','-Dio.netty.tryReflectionSetAccessible=true')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [56]:
from pyspark.sql.functions import *
from typing import Any

In [69]:
def convertDate(dt:Any,dformat:str='%Y-%m-%d') -> str:
    if isinstance(dt,str):
        converted = dateutil.parser.parse(dt)
    elif isinstance(dt,datetime.datetime):
        converted = dt
    elif isinstance(dt,(int,float)):
        converted = dateutil.parser.parse(str(int(dt)))
    else:
        return 'Unrecognized format'
    return str(converted.strftime(dformat))

def covertDatePandas(df:pd.Series)->pd.Series:
    return df.apply(convertDate)

converterScala  = udf(convertDate,returnType=StringType()) 
converterVector = pandas_udf(covertDatePandas,returnType=StringType())

def udf_date(dformat:str='%Y-%m-%d',vector:bool=False):
    return udf(lambda dt: converterVector(dt, dformat) if vector else converterScala)

In [75]:
def convertDate(dt:Any,dformat:str='%Y-%m-%d') -> Any:
    if isinstance(dt,str):
        converted = dateutil.parser.parse(dt)
    elif isinstance(dt,datetime.datetime):
        converted = dt
    elif isinstance(dt,(int,float)):
        converted = dateutil.parser.parse(str(int(dt)))
    else:
        return 'Unrecognized format'
    return converted.strftime(dformat)

def covertDatePandas(df:pd.Series)->pd.Series:
    return df.apply(convertDate)

converter = pandas_udf(covertDatePandas,returnType=StringType())    

def udf_date(dformat:str='%Y-%m-%d'):
    return udf(lambda dt: converter(dt, dformat))

In [ ]:
{
    'ColName':'OrderDate',
    'NewColumn':'modifiedDate',
    ''
}

In [116]:
def convertDate(df:DataFrame,
                cols = {'OrderDate':'modifiedDate'}
                ,dformat:str='%Y-%m-%d',
                vector:bool=True)->DataFrame:
    
    def _(dt:Any)->str:
        if isinstance(dt,str):
            converted = dateutil.parser.parse(dt)
        elif isinstance(dt,datetime.datetime):
            converted = dt
        elif isinstance(dt,(int,float)):
            converted = dateutil.parser.parse(str(int(dt)))
        else:
            return 'Unrecognized format'
        return converted.strftime(dformat)
    
    def covertDatePandas(df:pd.Series)->pd.Series:
        return df.apply(_)
    
    converterVector = pandas_udf(covertDatePandas,returnType=StringType())
    converterScala  = udf(_,returnType=StringType()) 
    
    reqUDF = lambda vector : converterVector if vector else converterScala
    
    return df.withColumn('vector',reqUDF(vector)(col('OrderDate')))
    

In [115]:
vector = False

def x():
    print(1)

def y():
    print(2)
    
z = lambda vector : x if vector else y

z(vector)()

2


In [103]:
#data.show()

In [117]:
convertDate(data,vector=True).show()

+-----------+--------------------+----------+---------+-------------+-------+----------+
|    orderID|           OrderDate|OrderValue|OrderType|OrderCategory|   city|    vector|
+-----------+--------------------+----------+---------+-------------+-------+----------+
|ORDLBWA5119|2004-10-02 09:28:...|   8135.52|      COD|         Home| durham|2004-10-02|
|ORDJQOW0469|2013-08-30 12:04:...|   6013.05|      COD|         Home|toronto|2013-08-30|
|ORDADHV6229|2017-09-13 20:33:...|   5115.79|      COD|      Furnish|toronto|2017-09-13|
|ORDOPWV1894|2004-03-03 15:36:...|   5361.52|      COD|  Electronics|toronto|2004-03-03|
|ORDENUU3212|2004-05-26 21:57:...|   2505.97|      COD|      Furnish| durham|2004-05-26|
|ORDKWLI8989|2013-05-30 07:26:...|   6577.39|      COD|         Home|halifax|2013-05-30|
|ORDNTEZ1527|2013-02-18 01:17:...|    5928.3|      COD|      Furnish|halifax|2013-02-18|
|ORDLNTY3345|2008-10-03 02:34:...|   6103.64|      COD|      Furnish| durham|2008-10-03|
|ORDXYPI3010|2008-09-

In [77]:
from pyspark.sql import functions as F
data.withColumn('test',converter(F.col('OrderDate'))).show()

+-----------+--------------------+----------+---------+-------------+-------+----------+
|    orderID|           OrderDate|OrderValue|OrderType|OrderCategory|   city|      test|
+-----------+--------------------+----------+---------+-------------+-------+----------+
|ORDLBWA5119|2004-10-02 09:28:...|   8135.52|      COD|         Home| durham|2004-10-02|
|ORDJQOW0469|2013-08-30 12:04:...|   6013.05|      COD|         Home|toronto|2013-08-30|
|ORDADHV6229|2017-09-13 20:33:...|   5115.79|      COD|      Furnish|toronto|2017-09-13|
|ORDOPWV1894|2004-03-03 15:36:...|   5361.52|      COD|  Electronics|toronto|2004-03-03|
|ORDENUU3212|2004-05-26 21:57:...|   2505.97|      COD|      Furnish| durham|2004-05-26|
|ORDKWLI8989|2013-05-30 07:26:...|   6577.39|      COD|         Home|halifax|2013-05-30|
|ORDNTEZ1527|2013-02-18 01:17:...|    5928.3|      COD|      Furnish|halifax|2013-02-18|
|ORDLNTY3345|2008-10-03 02:34:...|   6103.64|      COD|      Furnish| durham|2008-10-03|
|ORDXYPI3010|2008-09-

In [177]:
class Orca(object):
    def __init__(self,df):
        self.df = df
    def toSpark(self):
        return self.df
    def convertDate(self,
                    column:str,
                    newName:str=None
                ,dformat:str='%Y-%m-%d',
                addDays=0,
                getMonthFirst:bool=False,
                getMonthLast:bool=False,
                replaceBy:int=None,
                vector:bool=True)->DataFrame:
    
        def _(dt:Any)->str:
            if isinstance(dt,str):
                converted = dateutil.parser.parse(dt)
            elif isinstance(dt,datetime.datetime):
                converted = dt
            elif isinstance(dt,(int,float)):
                converted = dateutil.parser.parse(str(int(dt)))
            else:
                return 'Unrecognized format'
            if getMonthFirst:
                converted = converted.replace(day=1)
            if getMonthLast:
                next_month = converted.replace(day=28) + datetime.timedelta(days=4)
                converted  = next_month - datetime.timedelta(days=next_month.day)
            if replaceBy:
                converted = converted.replace(day=replaceBy)
            converted = converted+datetime.timedelta(days=addDays)
            return converted.strftime(dformat)

        def covertDatePandas(df:pd.Series)->pd.Series:
            return df.apply(_)

        converterVector = pandas_udf(covertDatePandas,returnType=StringType())
        converterScala  = udf(_,returnType=StringType()) 

        reqUDF = converterVector if vector else converterScala
        name   = column if newName is None else newName
        return Orca(self.df.withColumn(name,reqUDF(col(column))))
    

In [173]:
data.printSchema()

root
 |-- orderID: string (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- OrderValue: double (nullable = true)
 |-- OrderType: string (nullable = true)
 |-- OrderCategory: string (nullable = true)
 |-- city: string (nullable = true)



In [178]:
od = Orca(data.select('OrderDate'))

In [183]:
test = od.convertDate('OrderDate','simpleDate',dformat='%Y/%m/%d').toSpark()

In [186]:
od = Orca(test.select('simpleDate').alias('OrderDate'))

In [190]:
od.toSpark().show()

+----------+
|simpleDate|
+----------+
|2004/10/02|
|2013/08/30|
|2017/09/13|
|2004/03/03|
|2004/05/26|
|2013/05/30|
|2013/02/18|
|2008/10/03|
|2008/09/09|
|2001/03/05|
|2002/08/27|
|2006/10/08|
|2015/12/30|
|2000/03/09|
|2000/06/03|
|2015/09/25|
|2005/10/05|
|2019/01/12|
|2010/04/22|
|2020/06/09|
+----------+
only showing top 20 rows



In [194]:
od.convertDate('simpleDate','y')\
  .convertDate('simpleDate','addDays',addDays=1)\
  .convertDate('simpleDate','monthFirst',dformat='%Y%m%d',getMonthFirst=True)\
  .convertDate('simpleDate','monthLast',getMonthLast=True)\
  .convertDate('simpleDate','replaceBy',replaceBy=15,dformat='%Y/%m/%d').toSpark().show()

+----------+----------+----------+----------+----------+----------+
|simpleDate|         y|   addDays|monthFirst| monthLast| replaceBy|
+----------+----------+----------+----------+----------+----------+
|2004/10/02|2004-10-02|2004-10-03|  20041001|2004-10-31|2004/10/15|
|2013/08/30|2013-08-30|2013-08-31|  20130801|2013-08-31|2013/08/15|
|2017/09/13|2017-09-13|2017-09-14|  20170901|2017-09-30|2017/09/15|
|2004/03/03|2004-03-03|2004-03-04|  20040301|2004-03-31|2004/03/15|
|2004/05/26|2004-05-26|2004-05-27|  20040501|2004-05-31|2004/05/15|
|2013/05/30|2013-05-30|2013-05-31|  20130501|2013-05-31|2013/05/15|
|2013/02/18|2013-02-18|2013-02-19|  20130201|2013-02-28|2013/02/15|
|2008/10/03|2008-10-03|2008-10-04|  20081001|2008-10-31|2008/10/15|
|2008/09/09|2008-09-09|2008-09-10|  20080901|2008-09-30|2008/09/15|
|2001/03/05|2001-03-05|2001-03-06|  20010301|2001-03-31|2001/03/15|
|2002/08/27|2002-08-27|2002-08-28|  20020801|2002-08-31|2002/08/15|
|2006/10/08|2006-10-08|2006-10-09|  20061001|200

In [7]:
data = spark.read.parquet(r'C:\work\python-packages\mimic\mimic\examples\orders.parquet')

In [8]:
pdf = pd.read_parquet(r'C:\work\python-packages\mimic\mimic\examples\orders.parquet')

In [50]:
import dateutil.parser
import datetime
from typing import Iterator
def convertDate(dt:pd.Series)->pd.Series:
    def _(dt):
        dformat:str='%Y-%m-%d'
        converted = dateutil.parser.parse(str(dt)) 
        return str(converted.strftime(dformat))
    return dt.apply(_)
        

In [51]:
ctdt = pandas_udf(convertDate,returnType=StringType())

In [54]:
%timeit data.withColumn('test',ctdt('OrderDate')).select('test').count()

117 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
pdf['OrderDate'].apply(convertDate)

AttributeError: 'Timestamp' object has no attribute 'apply'

In [167]:
todayDate = datetime.datetime(year=2020,month=1,day=26)


print(todayDate.replace(day=29))

2020-01-29 00:00:00


In [197]:
%timeit od.convertDate('simpleDate','y')\
  .convertDate('simpleDate','addDays',addDays=1)\
  .convertDate('simpleDate','monthFirst',dformat='%Y%m%d',getMonthFirst=True)\
  .convertDate('simpleDate','monthLast',getMonthLast=True)\
  .convertDate('simpleDate','replaceBy',replaceBy=15,dformat='%Y/%m/%d').toSpark().show()

+----------+----------+----------+----------+----------+----------+
|simpleDate|         y|   addDays|monthFirst| monthLast| replaceBy|
+----------+----------+----------+----------+----------+----------+
|2004/10/02|2004-10-02|2004-10-03|  20041001|2004-10-31|2004/10/15|
|2013/08/30|2013-08-30|2013-08-31|  20130801|2013-08-31|2013/08/15|
|2017/09/13|2017-09-13|2017-09-14|  20170901|2017-09-30|2017/09/15|
|2004/03/03|2004-03-03|2004-03-04|  20040301|2004-03-31|2004/03/15|
|2004/05/26|2004-05-26|2004-05-27|  20040501|2004-05-31|2004/05/15|
|2013/05/30|2013-05-30|2013-05-31|  20130501|2013-05-31|2013/05/15|
|2013/02/18|2013-02-18|2013-02-19|  20130201|2013-02-28|2013/02/15|
|2008/10/03|2008-10-03|2008-10-04|  20081001|2008-10-31|2008/10/15|
|2008/09/09|2008-09-09|2008-09-10|  20080901|2008-09-30|2008/09/15|
|2001/03/05|2001-03-05|2001-03-06|  20010301|2001-03-31|2001/03/15|
|2002/08/27|2002-08-27|2002-08-28|  20020801|2002-08-31|2002/08/15|
|2006/10/08|2006-10-08|2006-10-09|  20061001|200